# SQLAlchemy w/ Textual SQL

SQLAlchemy at it's lowest level is just a wrapper around the DB api - a standard for database drivers in python like psycopg2 - a commonly used driver for PostgreSQL.

SQLAlchemy allows you to drop down into Textual sql to transfer over the wire when you'd like and it provides a handy `text` construct to do so in an injection-safe way. You can execute direct strings as well, but the `text` construct provides an easy api to use bound parameters to avoid sql injection attacks through raw string interpolations.

For more see: https://docs.sqlalchemy.org/en/14/core/tutorial.html#using-textual-sql

In [2]:
from sa2_swing.config import SQLALCHEMY_URI

from sqlalchemy import create_engine

engine = create_engine(SQLALCHEMY_URI)

## Create some tables

In [3]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute("""
    CREATE TABLE iF NOT EXISTS "user" (
        id serial primary key,
        first_name varchar(30) NOT NULL,
        last_name varchar(100)
    );
    """)


## Insert some data

In [3]:
from sqlalchemy import text

with engine.connect() as conn:
	conn.execute('TRUNCATE TABLE "user";')
	conn.execute(
		text("""INSERT INTO "user" (first_name, last_name) VALUES (:first_name, :last_name)"""),
		[
			{"first_name": "Collin", "last_name": "Choy"},
			{"first_name": "Jeff", "last_name": "Bridges"},
			{"first_name": "Tim", "last_name": None}
		]
	)

## Query some data

In [4]:
def select_all_users():
	with engine.connect() as conn:
		result = conn.execute(text('SELECT * FROM "user"'))
		return result.all()

select_all_users()

[(7, 'Collin', 'Choy'), (9, 'Tim', 'None'), (8, 'Jeff', 'Goldblum')]

In [6]:
stmt = text('SELECT first_name FROM "user" WHERE "user".first_name BETWEEN :start AND :end')
stmt = stmt.bindparams(start="A", end="M")

with engine.connect() as conn:
	result = conn.execute(stmt).fetchall()

result

[('Collin',), ('Jeff',)]

With type coercion

In [7]:
from sqlalchemy import bindparam, String

stmt = text('SELECT * FROM "user" WHERE "user".first_name BETWEEN :start AND :end')
stmt = stmt.bindparams(bindparam("start", type_=String), bindparam("end", type_=String))
with engine.connect() as conn:
	result = conn.execute(stmt, {"start": "A", "end": "D"}).all()
result

[(7, 'Collin', 'Choy')]

## Update some data

In [8]:
stmt = text("""
	UPDATE "user"
	SET last_name='Goldblum'
	WHERE first_name='Jeff'
""")
with engine.connect() as conn:
	result = conn.execute(stmt)

with engine.connect() as conn:
	result = conn.execute(text('select * from "user"')).all()
result

[(7, 'Collin', 'Choy'), (9, 'Tim', 'None'), (8, 'Jeff', 'Goldblum')]

## Delete some data

In [9]:
with engine.connect() as conn:
	conn.execute(text('DELETE from "user" where first_name = :first_name'), {"first_name": "Tim"})

select_all_users()

[(7, 'Collin', 'Choy'), (8, 'Jeff', 'Goldblum')]

## Cleanup: Drop tables

In [10]:
with engine.connect() as conn:
	conn.execute(text("""DROP TABLE "user";"""))